In [ ]:
!pip3 install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00


In [2]:
import requests
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def find_wiki_id(item):
  try:
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    return data['search'][0]['id']
  except:
    return None

In [4]:
def query_by_o(wid):
  query = """
      SELECT ?ssLabel ?wdLabel {
      VALUES (?o) {(wd:""" + wid + """)}
      ?s ?wdt ?o .
      ?wd wikibase:directClaim ?wdt .
      ?wd rdfs:label ?wdLabel .
      OPTIONAL {
      ?s rdfs:label ?sLabel .
      FILTER (lang(?sLabel) = "en")
      }
      FILTER (lang(?wdLabel) = "en")
      BIND (COALESCE(?sLabel, ?s) AS ?ssLabel)
      } 
      ORDER BY xsd:integer(STRAFTER(STR(?wd), "http://www.wikidata.org/entity/P"))
"""
  return query

In [5]:
def query_by_o_top(wid, count):
  query = """
      SELECT ?ssLabel ?wdLabel {
      VALUES (?o) {(wd:""" + wid + """)}
      ?s ?wdt ?o .
      ?wd wikibase:directClaim ?wdt .
      ?wd rdfs:label ?wdLabel .
      OPTIONAL {
      ?s rdfs:label ?sLabel .
      FILTER (lang(?sLabel) = "en")
      }
      FILTER (lang(?wdLabel) = "en")
      BIND (COALESCE(?sLabel, ?s) AS ?ssLabel)
      } 
      LIMIT """ + count
  return query

In [ ]:
def query_by_s(wid):
  query = """
    SELECT ?wdLabel ?ooLabel
    WHERE {
    VALUES (?s) {(wd:""" + wid + """)}
    ?s ?wdt ?o .
    ?wd wikibase:directClaim ?wdt .
    ?wd rdfs:label ?wdLabel .
    OPTIONAL {
    ?o rdfs:label ?oLabel .
    FILTER (lang(?oLabel) = "en")
    }
    FILTER (lang(?wdLabel) = "en")
    BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
    } 
"""
  return query

In [6]:
def query_by_s_top(wid, count):
  query = """
    SELECT ?wdLabel ?ooLabel
    WHERE {
    VALUES (?s) {(wd:""" + wid + """)}
    ?s ?wdt ?o .
    ?wd wikibase:directClaim ?wdt .
    ?wd rdfs:label ?wdLabel .
    OPTIONAL {
    ?o rdfs:label ?oLabel .
    FILTER (lang(?oLabel) = "en")
    }
    FILTER (lang(?wdLabel) = "en")
    BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
    } 
    LIMIT """ + count
  return query

In [1]:
query = """
    SELECT ?wdLabel ?ooLabel
    WHERE {
    VALUES (?s) {(wd:Q3884)}
    ?s ?wdt ?o .
    ?wd wikibase:directClaim ?wdt .
    ?wd rdfs:label ?wdLabel .
    OPTIONAL {
    ?o rdfs:label ?oLabel .
    FILTER (lang(?oLabel) = "en")
    }
    FILTER (lang(?wdLabel) = "en")
    BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
    } 
    LIMIT """ + '10'

In [2]:
query

'\n    SELECT ?wdLabel ?ooLabel\n    WHERE {\n    VALUES (?s) {(wd:Q3884)}\n    ?s ?wdt ?o .\n    ?wd wikibase:directClaim ?wdt .\n    ?wd rdfs:label ?wdLabel .\n    OPTIONAL {\n    ?o rdfs:label ?oLabel .\n    FILTER (lang(?oLabel) = "en")\n    }\n    FILTER (lang(?wdLabel) = "en")\n    BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)\n    } \n    LIMIT 10'

In [7]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


# results = get_results(url, query)

# for result in results["results"]["bindings"]:
#     print(result)

In [8]:
endpoint_url = "https://query.wikidata.org/sparql"

In [9]:
TEI_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg.csv"
spacy_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/spacy_full.csv"
GCP_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/GCP_kg.csv"

In [ ]:
# for subject in spo.csv
# wikiID = find_wiki_id(subject)
# if wikiID:
  # url_by_s = query_by_s(wikiID)
  # url_by_o = query_by_o(wikiID)

  # enrichment = get_results(url_by_s)
  # for entry in enrichment["results"]["bindings"]:
  # if predicate is not uri, 
    # predicate, object = entry['wdLabel']['value'],entry['ooLabel']['value']
    # generate s,p,o entry by wikidata

   # enrichment = get_results(url_by_o)
  # for entry in enrichment["results"]["bindings"]:
  # if predicate is not uri, 
    # predicate, subject = entry['wdLabel']['value'],entry['ssLabel']['value']
    # generate s,p,o entry by wikidata
  
# same for objects in spo.csv

In [10]:
def generate_enrichment(root_term, s_flag = True):
   root_id = find_wiki_id(root_term)
   if not root_id:
    #  print("term not found")
     return None
   if s_flag:
     q = query_by_s(root_id)
   else:
     q = query_by_o(root_id)
   
   return get_results(endpoint_url, q)
  #  for entry in enrichment["results"]["bindings"]:
  #     predicate, obj = entry['wdLabel']['value'],entry['ooLabel']['value']
  #     enrich_df = pd.DataFrame(list(zip(predicate, obj)),
  #              columns =['Predicate', 'Object'])
  #     enrich_df['Subject'] = root_term
  #  return enrich_df



In [11]:
def generate_enrichment_top(root_term, s_flag = True):
   root_id = find_wiki_id(root_term)
   if not root_id:
    #  print("term not found")
     return None
   if s_flag:
     q = query_by_s_top(root_id, '10')
   else:
     q = query_by_o_top(root_id, '10')
   
   return get_results(endpoint_url, q)
  #  for entry in enrichment["results"]["bindings"]:
  #     predicate, obj = entry['wdLabel']['value'],entry['ooLabel']['value']
  #     enrich_df = pd.DataFrame(list(zip(predicate, obj)),
  #              columns =['Predicate', 'Object'])
  #     enrich_df['Subject'] = root_term
  #  return enrich_df



### TEI enrichment, only objects, because subjects are all chapter titles

In [ ]:
# read csv file
import pandas as pd
o = set()
tb_kg = pd.read_csv(TEI_file, header=None)
for index, row in tb_kg.iterrows():
  # s = row[0]
  obj = row[2]
  o.add(obj)

In [ ]:
len(o)

1176

In [ ]:
filter_o = set()
for term in o:
  if term[0].isalpha():
    filter_o.add(term)

In [ ]:
len(filter_o)

1035

In [ ]:
print("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI_wiki_{}.csv".format(0))

/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI_wiki_0.csv


In [ ]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in filter_o:
  count += 1
  enrichment = generate_enrichment(term, False)
  if enrichment:

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count % 100 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, sub = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_{}.csv".format(subset))



term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not f

In [ ]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset = 11
for term in filter_o:
  count += 1
  enrichment = generate_enrichment(term, True)
  if enrichment:

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, o = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(o)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count % 100 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, obj = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_{}.csv".format(subset))



term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not found
term not f

### GCP

In [18]:
import pandas as pd
tb_kg = pd.read_csv(GCP_file)
node_gcp = set()
for index, row in tb_kg.iterrows():
  e = row[0]
  t = e.split(':')[1][:-1]
  node_gcp.add(t)

In [ ]:
len(node_gcp)

9380

In [ ]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in node_gcp:
  enrichment = generate_enrichment_top(term, False)
  if enrichment:
    count += 1

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count % 1000 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, sub = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_{}.csv".format(subset))


In [16]:
full_df.head(10)

,Predicate,Subject,Object
0,place of birth,Josie Maran,Menlo Park
1,place of birth,Henry Cowell,Menlo Park
2,place of birth,Mary Callahan Erdoes,Menlo Park
3,place of birth,Ariel Rittenhouse,Menlo Park
4,place of birth,Robert Rich,Menlo Park
5,place of birth,Taylor Eigsti,Menlo Park
6,place of birth,Alex Weed,Menlo Park
7,place of birth,Djay Brawner,Menlo Park
8,place of birth,David Vestal,Menlo Park
9,place of birth,Joe Wise,Menlo Park


In [20]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset += 1
for term in node_gcp:
  enrichment = generate_enrichment_top(term, True)
  if enrichment:
    count += 1


    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, o = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(o)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count % 1000 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, obj = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_{}.csv".format(subset))


### spacy

In [12]:
# read csv file
import pandas as pd
ent = set()
tb_kg = pd.read_csv(spacy_file)
for index, row in tb_kg.iterrows():
  s = row['source']
  obj = row['target']
  ent.add(s)
  ent.add(obj)

In [13]:
len(ent)

6729

In [14]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in ent:
  enrichment = generate_enrichment_top(term, False)
  if enrichment:
    count += 1

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count!=0 and count % 1000 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, sub = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_{}.csv".format(subset))


In [15]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset += 1
for term in ent:
  enrichment = generate_enrichment_top(term, True)
  if enrichment:
    count += 1

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(s)
      
    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])
  
  if count!=0 and count % 1000 == 0:
    full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_{}.csv".format(subset))
    full_df = pd.DataFrame()
    predicate, obj = [], []
    subset += 1

full_df.to_csv("/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_{}.csv".format(subset))


# Analysis - spaCy enrich

In [1]:
import glob
csvfilepath = '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/'
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/spacy/spacy_wiki_7.csv']

In [2]:
import pandas as pd
spacy_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
spacy_wiki.tail()

,Unnamed: 0,Predicate,Subject,Object
13259104,7210,WordNet 3.1 Synset ID,Invocation,05987026-n
13259105,7211,WordNet 3.1 Synset ID,Invocation,07204950-n
13259106,7212,Microsoft Academic ID,Invocation,2776527387
13259107,7213,GND ID,Invocation,4291290-8
13259108,7214,National Library of Israel J9U ID,Invocation,987007558172005171


In [12]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/spacy_full.csv')

In [30]:
node_spacy = set()
for index, row in df.iterrows():
  s = row['source']
  t = row['target']

  node_spacy.add(s)
  node_spacy.add(t)

In [46]:
node_spacy = set()
edge_spacy = set()
for index, row in spacy_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']


  node_spacy.add(s)
  node_spacy.add(t)
  edge_spacy.add(e)

In [5]:
spacy_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/spacy_enriched.csv')

In [50]:
spacy_wiki['Predicate'].describe()

count        13259109
unique           1063
top       instance of
freq          2154956
Name: Predicate, dtype: object

In [6]:
tei_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg.csv', header=None)

In [43]:
node = set()
for index, row in tei_df.iterrows():
  s = row[0]
  t = row[2]

  node.add(s)
  node.add(t)

In [47]:
diff_node = set()
for n in node_spacy:
  if n in node:
  # if n in node:
    diff_node.add(n)

In [48]:
len(node_spacy), len(edge_spacy), len(diff_node)

(22238, 1063, 114)

In [13]:
df.head()

,Unnamed: 0,source,edge,target
0,0,computing,based_on,computing paradigm ethics
1,1,multiple services,stored_on_multiple,several organizations
2,2,Systems,span,multiple security process
3,3,border,becomes,accountability
4,4,something,make,case


In [22]:
cloud_comp1 = df[(df['source']=='cloud computing')] 

In [23]:
cloud_comp2 = df[(df['target']=='cloud computing')] 


In [24]:
cloud_comp = pd.concat([cloud_comp1, cloud_comp2])

In [25]:
cloud_comp = cloud_comp.drop(columns='Unnamed: 0')


In [32]:
cloud_comp.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/spacy.csv')

In [34]:
cloud_comp1 = spacy_wiki[(spacy_wiki['Subject']=='cloud computing')] 

In [26]:
cloud_comp1

,Unnamed: 0,source,edge,target
254,254,cloud computing,grouped_into_several,general categories
265,265,cloud computing,are,technological breakthroughs


In [27]:
cloud_comp2 = spacy_wiki[(spacy_wiki['Object']=='cloud computing')] 

In [28]:
cloud_comp2

,Unnamed: 0,Predicate,Subject,Object
5049631,0,field of work,content management,cloud computing
5049632,1,named after,cacoxenite,cloud computing
5049633,2,has part(s),hospitality,cloud computing
5049634,3,subclass of,http://www.wikidata.org/entity/Q11959969,cloud computing
5049635,4,different from,expense,cloud computing
...,...,...,...,...
13253433,1539,industry,Invocation,cloud computing
13255617,3723,subclass of,Invocation,cloud computing
13256091,4197,subclass of,Invocation,cloud computing
13258204,6310,subclass of,Invocation,cloud computing


In [39]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')
cloud_comp_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/spacy_wiki.csv')

In [29]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')

In [31]:
node_spacy_wiki = set()
for index, row in cloud_comp_wiki.iterrows():
  s = row['source']
  t = row['target']

  node_spacy_wiki.add(s)
  node_spacy_wiki.add(t)

In [35]:
lead_back=[]
for x in node_spacy_wiki:
  if x in node_spacy_wiki and x != 'cloud computing':
    lead_back.append(x)

In [36]:
len(lead_back)

3

In [34]:
lead_back

[nan, 'technological  breakthroughs', 'general  categories']

# Analysis - GCP enrich

In [34]:
import glob
csvfilepath = '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/'
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_7.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_8.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/GCP/GCP_wiki_9.csv',
 '/content/drive/MyDrive/MSCS Project/CS

In [35]:
import pandas as pd
gcp_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
gcp_wiki.tail()

<ipython-input-35-7f74dee6c8dd>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  gcp_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)


,Unnamed: 0,Predicate,Subject,Object
48680433,4738,found in taxon,GET,Homo sapiens
48680434,4739,ortholog,GET,Resf1
48680435,4740,subclass of,GET,protein-coding gene
48680436,4741,encodes,GET,Retroelement silencing factor 1
48680437,4742,encodes,GET,Uncharacterized protein KIAA1551


In [36]:
gcp_wiki = gcp_wiki.drop(columns='Unnamed: 0')


In [52]:
gcp_wiki['Predicate'].describe()

count        48680438
unique           1740
top       instance of
freq          6568695
Name: Predicate, dtype: object

In [6]:
cloud_comp1 = gcp_wiki[(gcp_wiki['Subject']=='cloud computing')] 
cloud_comp2 = gcp_wiki[(gcp_wiki['Object']=='cloud computing')] 


In [7]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')
cloud_comp_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/gcp_wiki.csv')

In [8]:
cloud_comp_wiki.head(10)

,Predicate,Subject,Object
178501,has quality,cloud computing,Scalability
180092,has quality,cloud computing,Outputs
181683,has quality,cloud computing,71
183284,has quality,cloud computing,9.6
184885,has quality,cloud computing,information problem
186486,has quality,cloud computing,cues
188097,has quality,cloud computing,MC68000
189718,has quality,cloud computing,integers
191349,has quality,cloud computing,expectations
192988,has quality,cloud computing,1994


In [37]:
gcp_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/GCP_kg.csv')

In [38]:
node_gcp = set()
for index, row in gcp_df.iterrows():
  e = row[0]
  t = e.split(':')[1][:-1]
  node_gcp.add(t)

In [40]:
gcp_node_wiki = set()
gcp_edge_wiki = set()
for index, row in gcp_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']

  gcp_node_wiki.add(t)
  gcp_node_wiki.add(s)
  gcp_edge_wiki.add(e)


In [41]:
overlap_node_gcp = set()
for n in gcp_node_wiki:
  if n in node_gcp:
    overlap_node_gcp.add(n)

In [42]:
len(gcp_node_wiki), len(gcp_edge_wiki), len(overlap_node_gcp)

(69246, 1740, 6538)

In [15]:
lead_back=[]
for x in node_wiki:
if x in node_gcp and x != 'cloud computing':
    lead_back.append(x)


In [16]:
len(lead_back)

5572

# Analysis - TEI

In [1]:
import glob
csvfilepath = '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/'
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_7.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_8.csv',
 '/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_enrichment/TEI/TEI_wiki_9.csv',
 '/content/drive/MyDrive/MSCS Project/CS

In [2]:
import pandas as pd
tei_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
tei_wiki.tail()

,Unnamed: 0,Predicate,Subject,Object
3541492,414,author name string,Fabric controller,Samir Suweis
3541493,415,author name string,Fabric controller,Chengyi Tu
3541494,416,publication date,Fabric controller,2019-01-01T00:00:00Z
3541495,417,title,Fabric controller,Critical Slowing down Associated with Critical...
3541496,418,ScienceDirect topic ID,Fabric controller,computer-science/fabric-controller


In [3]:
tei_wiki = tei_wiki.drop(columns='Unnamed: 0')


In [4]:
tei_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/tei_wiki.csv')

In [53]:
tei_wiki['Predicate'].describe()

count          3541497
unique            1089
top       main subject
freq            954871
Name: Predicate, dtype: object

In [22]:
tei_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg.csv', header=None)

In [26]:
node = set()
for index, row in tei_df.iterrows():
  s = row[0]
  t = row[2]

  node.add(s)
  node.add(t)

In [27]:
len(node)

1187

In [30]:
node_tei = set()
edge_tei = set()
for index, row in tei_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']


  node_tei.add(s)
  node_tei.add(t)
  edge_tei.add(e)

In [31]:
len(node_tei), len(edge_tei)

(159887, 1089)

In [32]:
overlap_node = set()
for n in node_tei:
  if n in node:
    overlap_node.add(n)

In [33]:
len(overlap_node)

429